In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]: # 다섯개만 출력 
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [24]:
from PIL import Image # 파이썬 이미지 라이브러리, 이미지 출력 확인

Image.open('/kaggle/input/microsoft-rice-disease-classification-challenge/Images/id_004wknd7qd.jpg').size

In [25]:
Image.open('/kaggle/input/microsoft-rice-disease-classification-challenge/Images/id_004wknd7qd.jpg')

In [26]:
train = pd.read_csv('/kaggle/input/microsoft-rice-disease-classification-challenge/Train.csv')
train

In [27]:
train.iloc[0::2, :]

In [28]:
train = train.iloc[0::2, :].copy()
train

In [29]:
train['Image_id'].value_counts()

In [30]:
test = pd.read_csv('/kaggle/input/microsoft-rice-disease-classification-challenge/Test.csv')
test

In [31]:
test = test.iloc[0::2, :].copy()
test

In [32]:
train['path'] = '/kaggle/input/microsoft-rice-disease-classification-challenge/Images/' + train['Image_id']
train

In [33]:
Image.open('/kaggle/input/microsoft-rice-disease-classification-challenge/Images/id_zzt8y9q0x0_rgn.jpg')

In [34]:
Image.open('/kaggle/input/microsoft-rice-disease-classification-challenge/Images/id_zzt8y9q0x0.jpg')

In [35]:
# 주소 생성
test['path'] = '/kaggle/input/microsoft-rice-disease-classification-challenge/Images/' + test['Image_id']
test

In [36]:
pd.options.display.max_colwidth = 99 

In [37]:
## (500, 400)인 이미지 사이즈를 줄이기 : 256 + 64 = 320
os.makedirs('train_320', exist_ok=True) # 폴더 생성 -> 이미지를 새롭게 저장 미리 사이즈 줄여서 저장 할거야 

In [38]:
import cv2
import shutil
from tqdm import tqdm

for path in tqdm(train['path']):
    image = cv2.imread(path) # 이미지 읽기 (주소넣기)
    image = cv2.resize(image, (320, 320)) # 이미지를 320으로 줄임. 
    
    cv2.imwrite('train_320/' + path.split('/')[-1], image) # 이미지 저장 
    
shutil.make_archive('train_320' ,'zip','train_320') # 압축 

In [39]:
os.makedirs('test_320', exist_ok=True)

In [40]:
for path in tqdm(test['path']):
    image = cv2.imread(path)
    image = cv2.resize(image, (320, 320))
    
    cv2.imwrite('test_320/' + path.split('/')[-1], image)
    
shutil.make_archive('test_320' ,'zip','test_320')

In [41]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator # 이미지 전처리 라이브러리 
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *

from tensorflow.keras.applications.efficientnet import * # 딥러닝 문제 efficientent 

In [42]:
train['new_path'] = 'train_320/' + train['Image_id'] 
test['new_path'] = 'test_320/' + test['Image_id']

In [43]:
test

In [44]:
result = 0 # 모델이 예측을 여러번 할 거라서

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # n_splits=5 성능 

for train_idx, valid_idx in skf.split(train, train['Label']): # 정답클래스 비율 같게 
    X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
    
    es = EarlyStopping(patience=3, restore_best_weights=True) # 학습하다가 언젠간 멈추게 하는거 
    rl = ReduceLROnPlateau(patience=2, verbose=1) # 러닝레이트 : 학습 섬세하게 하기...
    
    idg = ImageDataGenerator(horizontal_flip=True) # 이미지 데이터 전처리 : 데이터 수가 부족하면 horizontal_flip=True 좌우반전 시켜 학습 가능
    idg2 = ImageDataGenerator() 
    
    train_generator = idg.flow_from_dataframe(X_train, x_col='new_path', y_col='Label', target_size=(320, 320), batch_size=32) # 전처리, 데이터셋/주소/정답컬럽/이미지사이즈/배치사이즈
    valid_generator = idg2.flow_from_dataframe(X_valid, x_col='new_path', y_col='Label', target_size=(320, 320), batch_size=32)
    
    model = Sequential()
    
    model.add(EfficientNetB1(pooling='avg', include_top=False))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam', metrics='acc', loss='categorical_crossentropy')
    
    model.fit(train_generator, validation_data=valid_generator, epochs=7, callbacks=[es, rl])
    
    test_generator = idg2.flow_from_dataframe(test, x_col='new_path', y_col=None, target_size=(320, 320), batch_size=32, shuffle=False, class_mode=None)
    
    result += model.predict(test_generator, workers=2, verbose=1) / 5

In [45]:
sub = pd.read_csv('/kaggle/input/microsoft-rice-disease-classification-challenge/SampleSubmission.csv')
sub

In [46]:
sub.iloc[:, 1:] = result

In [47]:
sub.to_csv('rice_sub.csv', index=0)

- 실험은 스스로 해보기
- 평가방식은 log-loss